Inferring a 10-node network
===============

SWING (Sliding Window Inference for Network Generation) is a network inference tool that incorporates elements from multivariate Granger causality and sliding window regression. This tutorial demonstrates how to infer a 10-node network using SWING-RF (RandomForest), SWING-PLSR (Partial Least Squares Regression), and SWING-Lasso. 

SWING has user-selected parameters that require knowledge of the system of interest (such as range of time-delays). 


- $k_{min}$ and $k_{max}$: $k_{min}$ and $k_{max}$ refer to the allowed delay range. If $k_{min} = 1$ and $k_{min} = 3$, we are assuming the responses are delayed by at least one timepoint and at most 3 timepoints. We recommend you set values for k_min and k_max using prior knowledge or delay analysis such as cross correlation.  